**写在前面**：本节内容是 [Datawhale三月的组队学习 - 集成学习（上）- CH2-机器学习基础模型回顾 -【Task3 掌握偏差与方差理论】](https://github.com/datawhalechina/team-learning-data-mining/blob/master/EnsembleLearning/CH2-%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E5%9F%BA%E7%A1%80%E6%A8%A1%E5%9E%8B%E5%9B%9E%E9%A1%BE/%E7%AC%AC%E4%BA%8C%E7%AB%A0%EF%BC%9A%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0%E5%9F%BA%E7%A1%80.ipynb) 的学习笔记，对应notebook的2.1(4)节，学习周期2天（后调整为4天）

## 导入库和数据

In [21]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm_notebook as tqdm
import sklearn
from sklearn import datasets
from sklearn.model_selection  import train_test_split
import xgboost
# Day1
from mlxtend.evaluate import bias_variance_decomp # 偏差-方差分解
from sklearn.linear_model import LassoLarsIC # Lasso，不清楚具体是什么
from sklearn.metrics import mean_squared_error as MSE
# Day2
from sklearn.model_selection import KFold # K折交叉验证
from sklearn.model_selection import LeaveOneOut # 留一法
from sklearn.model_selection import StratifiedKFold # 分层交叉验证
from sklearn.model_selection import LeavePOut # 留P交叉验证
from sklearn.model_selection import GroupKFold # 分组交叉验证
import random
from sklearn.linear_model import Lasso,Ridge,LinearRegression
from sklearn.metrics import mean_absolute_error as MAE
# Day4
from sklearn.decomposition import PCA
from sklearn.decomposition import LatentDirichletAllocation as LDA # 并不是线性判别分析
from sklearn.decomposition import TruncatedSVD
# 测试用
import numba

In [22]:
# boston数据集作为本笔记的回归实验数据
boston = datasets.load_boston()
Xb = boston.data
yb = boston.target
features = boston.feature_names
boston_data = pd.DataFrame(Xb,columns=features)
boston_data["Price"] = yb

X_train, X_test, y_train, y_test = train_test_split(Xb,yb,test_size=0.25)

In [23]:
# breast_cancer数据集作为本笔记的分类实验数据
breast_cancer = datasets.load_breast_cancer()
Xbc = breast_cancer.data
ybc = breast_cancer.target
features = breast_cancer.feature_names
breast_cancer = pd.DataFrame(Xbc,columns=features)
breast_cancer["target"] = ybc

In [24]:
# 使用xgboost的回归模型作为实验模型
model = xgboost.XGBRegressor()
model.fit(X=X_train, y=y_train)
y_pred = model.predict(X_test)
print('MSE:\t%.5f' % MSE(y_test, y_pred))

MSE:	15.74817


## Day1

### 基础概念
**参考资料**：
- [偏差和方差有什么区别？](https://www.zhihu.com/question/20448464/answer/765401873)
- 微信公众号:数据派THU —《干货 ：教你用Python来计算偏差-方差权衡》
- [理解赤池信息量（AIC）,贝叶斯信息量（BIC）](https://blog.csdn.net/chieryu/article/details/51746554)
- [模型选择方法：AIC和BIC](https://www.jianshu.com/p/4c8cf5df2092)
- [AIC, BIC 和 L1,L2 等正则化有什么区别？](https://zhuanlan.zhihu.com/p/26372789)
- [误差与残差](https://blog.csdn.net/fwj_ntu/article/details/82697433)
- [残差、方差、偏差、MSE均方误差、Bagging、Boosting、过拟合欠拟合和交叉验证](https://blog.csdn.net/u010986753/article/details/102495494)

#### 概念理解

- **偏差**：bias  
    指**预测结果**与**真实值**之间的`差异`，排除噪声的影响。反映模型本身的精确度。
        偏差更多的是针对某个模型输出的样本误差，偏差是模型无法准确表达数据关系导致，比如模型过于简单，非线性的数据关系采用线性模型建模，偏差较大的模型是错的模型


- **方差**：variance  
    指多个(次)模型**输出的结果**之间的`离散差异`。反映模型的稳定性。
        注意这里写的是多个模型或者多次模型，即不同模型或同一模型不同时间的输出结果方差较大，方差是由训练集的数据不够导致。一方面量 【数据量】不够，有限的数据集过度训练导致模型复杂；另一方面质【样本质量】不行，测试集中的数据分布未在训练集中，导致每次抽样训练模型时，每次模型参数不同，输出的结果都无法准确的预测出正确结果  


- **偏差-方差分解**  
可证得：(我还没证)
   $$
   E\left(y_{0}-\hat{f}\left(x_{0}\right)\right)^{2}=\operatorname{Var}\left(\hat{f}\left(x_{0}\right)\right)+\left[\operatorname{Bias}\left(\hat{f}\left(x_{0}\right)\right)\right]^{2}+\operatorname{Var}(\varepsilon)
   $$      


- **误差**：Errors  
    指**观测值**与**真实值**的偏差。  


- **残差**：Residuals  
    指**估计值**与**观测值**的偏差。如果回归模型正确的话， 我们可以将残差看作误差的观测值。  


- **残差、方差、偏差总结**
    - **简单模型**：偏差大，方差小（简单模型受样本值的影响较小，稳定性高），容易造成欠拟合
    - **复杂模型**：偏差小，方差大，容易产生过拟合
    - 判断偏差大还是方差大：
        1. 模型上的训练样本的真实值较少，则偏差大（欠拟合）
        2. 在训练样本上样本拟合的较好，但在测试集上拟合较差，则方差大（过拟合Overfiting）
        3. 当偏差较大时，表示目标可能未在模型上（即未瞄准靶心），需要重新训练model（有可能未考虑其他因素对样本的影响，或者应让模型更复杂考虑更高次幂的情况）。增加网络层数，增加隐藏层神经元数量，增加算法迭代次数，或者用更好的优化算法。
        4. 当方差较大时：增加更多的数据或正则化  
        
- **AIC赤池信息量**  
越小越好。也写做：-2ln(L) + 2k
$$AIC = \frac{1}{d\hat{\sigma}^2}(RSS  +  2d\hat{\sigma}^2)$$      
- **BIC贝叶斯信息量**  
也写做：-2ln(L) + ln(n)*k
$$BIC = \frac{1}{n}(RSS + log(n)d\hat{\sigma}^2)$$    
    

#### 偏差-方差分解的mlxtend调用实现

In [15]:
mse,bias,var = bias_variance_decomp(model, X_train, y_train, X_test, y_test,loss='mse', num_rounds=200)
print('MSE:\t%.3f' % mse)
print('Bias:\t%.3f' % bias)
print('Var:\t%.3f' % var)

MSE:	15.416
Bias:	10.676
Var:	4.740


可以发现这里`MSE = Bias + Var`

## Day2

### 特征提取

#### 训练误差修正
**参考资料**:
- [aic python_使用python+sklearn实现Lasso 模型选择：交叉验证/ AIC / BIC](https://blog.csdn.net/weixin_35473090/article/details/112500913)

##### AIC赤池信息量准则

In [37]:
model_aic = LassoLarsIC(criterion='aic')
model_aic.fit(X_train, y_train)
y_pred = model_aic.predict(X_test)
print('AIC:\t%.5f' % model_aic.alpha_)
print('MSE:\t%.5f' % MSE(y_test, y_pred))

AIC:	0.00143
MSE:	31.18560


##### BIC贝叶斯信息量准则

In [38]:
model_bic = LassoLarsIC(criterion='bic')
model_bic.fit(X_train, y_train)
y_pred = model_bic.predict(X_test)
print('BIC:\t%.5f' % model_bic.alpha_)
print('MSE:\t%.5f' % MSE(y_test, y_pred))

BIC:	0.07306
MSE:	39.80559


#### 交叉验证：Cross Validation
**参考资料**：
- [数据集划分train_test_split\交叉验证Cross-validation](https://blog.csdn.net/u010986753/article/details/98069124)
- [sklearn.model_selection](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection)
- 《机器学习》 - 周志华

##### 简单交叉验证：train_test_split
通过反复重新选择训练集和测试集，继续训练数据和检验模型，最后选择损失函数评估最优的模型和参数。
- **好处**：  
    处理简单，只需随机把原始数据分为两组即可。
    
- **缺点**：  
    只进行了一次划分，数据结果具有偶然性，没有达到交叉的思想，由于是随机的将原始数据分组，所以最后验证集分类准确率的高低与原始数据的分组有很大的关系，得到的结果并不具有说服性。
    
**sklearn参数**：  
- `test_size`：测试集的样本比例或样本数量
- `shuffle`：拆分前是否对数据进行混洗

**注**：参数未全部列出，详细可见参考文档

In [146]:
X_train, X_test, y_train, y_test = train_test_split(Xbc, ybc, test_size=0.30,
                                                    shuffle=True, random_state=320)
a, b = len(y_train[y_train == 0]), len(y_train[y_train == 1])
c, d = len(y_test[y_test == 0]), len(y_test[y_test == 1])
print('train.length: %d\t, \ttest.length: %d' %
      (train_index.shape[0], test_index.shape[0]))
print('train==0: %d\t, train=1: %d\t, ratio: %.2f' % (a, b, a / b))
print('test==0:  %d\t, test=1:  %d\t, ratio: %.2f' % (c, d, c / d))

train.length: 568	, 	test.length: 1
train==0: 147	, train=1: 251	, ratio: 0.59
test==0:  65	, test=1:  106	, ratio: 0.61


##### 留一交叉验证 Leave-one-out Cross Validation
在数据缺乏的情况下使用，如果设原始数据有N个样本，那么LOO-CV就是N-CV，即每个样本单独作为验证集，其余的N-1个样本作为训练集，故LOO-CV会得到N个模型，用这N个模型最终的验证集的分类准确率的平均数作为此下LOO-CV分类器的性能指标。
- **优点**：  
    不存在数据分布不一致,每一回合中几乎所有的样本皆用于训练模型，因此最接近原始样本的分布，这样评估所得的结果比较可靠。实验过程中没有随机因素会影响实验数据，确保实验过程是可以被复制的。
- **缺点**：  
    耗时,计算成本高，需要建立的模型数量与原始数据样本数量相同。当数据集较大时几乎不能使用。

**sklearn参数**：  
- 无

In [132]:
LOO = LeaveOneOut()
for loop, (train_idx, test_idx) in enumerate(LOO.split(Xbc, ybc)):
    X_train, X_test = Xbc[train_idx], Xbc[test_idx]
    y_train, y_test = ybc[train_idx], ybc[test_idx]
#     a,b = len(y_train[y_train==0]),len(y_train[y_train==1])
#     c,d = len(y_test[y_test==0]),len(y_test[y_test==1])
#     print('################(%d)################' % loop)
#     print('train.length: %d, test.length: %d' % (train_idx.shape[0], test_idx.shape[0]))
#     print('train==0: %d\t, train=1: %d\t, ratio: %.2f' % (a,b,a/b))
#     print('test==0:  %d\t, test=1:  %d\t, ratio: %.2f' % (c,d,c/d))
else:
    print('loop:',loop)

loop: 568


##### 留P交叉验证 LeavePOut
留一法的变体。它从完整的数据集里删除 p 个样本，产生所有可能的训练集和检验集。对于 n个样本，能产生m个训练-检验对。

In [5]:
LPO= LeavePOut(p=300)
for loop, (train_idx, test_idx) in tqdm(enumerate(LPO.split(Xbc, ybc))):
    if loop>10000:
        break
    X_train, X_test = Xbc[train_idx], Xbc[test_idx]
    y_train, y_test = ybc[train_idx], ybc[test_idx]
#     a,b = len(y_train[y_train==0]),len(y_train[y_train==1])
#     c,d = len(y_test[y_test==0]),len(y_test[y_test==1])
#     print('################(%d)################' % loop)
#     print('train.length: %d, test.length: %d' % (train_idx.shape[0], test_idx.shape[0]))
#     print('train==0: %d\t, train=1: %d\t, ratio: %.2f' % (a,b,a/b))
#     print('test==0:  %d\t, test=1:  %d\t, ratio: %.2f' % (c,d,c/d))
else:
    print('loop:',loop)

0it [00:00, ?it/s]

##### 自助法 Bootstrapping
另一种比较特殊的交叉验证方式，也用于样本量少的时候。假设有m个样本（m较小），每次在这m个样本中随机采集一个样本，放入训练集，采样完后把样本放回。这样重复采集m次，我们得到m个样本组成的训练集。这m个样本中很有可能有重复的样本数据。同时，用没有被采样到的样本做测试集。这样接着进行交叉验证。由于我们的训练集有重复数据，这会改变数据的分布，因而训练结果会有估计偏差，因此，此种方法不是很常用，除非数据量真的很少，比如小于20个。
- **优点**：  
在数据集较小、难以划分时很有用，能从D中产生不同的S，对集成学习等方法有好处

- **缺点**：  
产生的S改变了D的分布，会引入估计偏差

In [20]:
data = pd.DataFrame(Xb)
target = pd.DataFrame(yb)
for loop in range(data.shape[0]):
    X_train = data.sample(frac=1.0,replace=True) # 有放回随机采样
    y_train = target.sample(frac=1.0,replace=True) 
    X_test = data.loc[X_train.index.difference(X_train.index)].copy() # 将未采样的样本作为测试集
    y_test = target.loc[y_train.index.difference(y_train.index)].copy()

##### K折交叉验证 K-Folder Cross Validation
应用最多，K-CV可以有效的避免过拟合与欠拟合的发生，最后得到的结果也比较具有说服性。
- **实现步骤**：
    1. 不重复抽样将原始数据随机分为 k 份。
    2. 每一次挑选其中 1 份作为测试集，剩余 k-1 份作为训练集用于模型训练。
    3. 重复(2) k 次，这样每个子集都有一次机会作为测试集，其余机会作为训练集。在每个训练集上训练后得到一个模型，用这个模型在相应的测试集上测试，计算并保存模型的评估指标，
    4. 计算 k 组测试结果的平均值作为模型精度的估计，并作为当前 k 折交叉验证下模型的性能指标。
  
  
- **优点**：  
    降低由一次随机划分带来的偶然性，提高其泛化能力，提高对数据的使用效率。
- **缺点**：  
    可能存在一种情况：数据集有5类，抽取出来的也正好是按照类别划分的5类，也就是说第一折全是0类，第二折全是1类，等等；这样的结果就会导致，模型训练时。没有学习到测试集中数据的特点，从而导致模型得分很低，甚至为0
    
**sklearn参数**：  
- `n_splits`：折数
- `shuffle`：拆分前是否对数据进行混洗

**注**：参数未全部列出，详细可见参考文档

In [189]:
KF = KFold(n_splits=3, shuffle=True, random_state=320)
for loop, (train_idx, test_idx) in enumerate(KF.split(Xbc, ybc)):
    if loop>10000:
        break
    X_train, X_test = Xbc[train_idx], Xbc[test_idx]
    y_train, y_test = ybc[train_idx], ybc[test_idx]
    a,b = len(y_train[y_train==0]),len(y_train[y_train==1])
    c,d = len(y_test[y_test==0]),len(y_test[y_test==1])
    print('######################(%d)######################' % loop)
    print('train.length: %d\t, \ttest.length: %d' % (train_idx.shape[0], test_idx.shape[0]))
    print('train==0: %d\t, train=1: %d\t, ratio: %.2f' % (a,b,a/b))
    print('test==0:  %d\t, test=1:  %d\t, ratio: %.2f' % (c,d,c/d))
else:
    print('loop:',loop)

######################(0)######################
train.length: 379	, 	test.length: 190
train==0: 139	, train=1: 240	, ratio: 0.58
test==0:  73	, test=1:  117	, ratio: 0.62
######################(1)######################
train.length: 379	, 	test.length: 190
train==0: 135	, train=1: 244	, ratio: 0.55
test==0:  77	, test=1:  113	, ratio: 0.68
######################(2)######################
train.length: 380	, 	test.length: 189
train==0: 150	, train=1: 230	, ratio: 0.65
test==0:  62	, test=1:  127	, ratio: 0.49
loop: 2


##### 分层交叉验证 StratifiedKFold
KFold的变体。对非平衡数据可以用分层采样，能够在每一份子集中都保持和原始数据集相同的类别比例。  

**sklearn参数**：  
- `n_splits`：折数
- `shuffle`：拆分前是否对数据进行混洗

**注**：参数未全部列出，详细可见参考文档

In [8]:
SKF = StratifiedKFold(n_splits=3, shuffle=True, random_state=320)
for loop, (train_idx, test_idx) in enumerate(SKF.split(Xbc, ybc)):
    X_train, X_test = Xbc[train_idx], Xbc[test_idx]
    y_train, y_test = ybc[train_idx], ybc[test_idx]
    a,b = len(y_train[y_train==0]),len(y_train[y_train==1])
    c,d = len(y_test[y_test==0]),len(y_test[y_test==1])
    print('######################(%d)######################' % loop)
    print('train.length: %d\t, \ttest.length: %d' % (train_idx.shape[0], test_idx.shape[0]))
    print('train==0: %d\t, train=1: %d\t, ratio: %.2f' % (a,b,a/b))
    print('test==0:  %d\t, test=1:  %d\t, ratio: %.2f' % (c,d,c/d))
else:
    print('loop:',loop)

######################(0)######################
train.length: 379	, 	test.length: 190
train==0: 141	, train=1: 238	, ratio: 0.59
test==0:  71	, test=1:  119	, ratio: 0.60
######################(1)######################
train.length: 379	, 	test.length: 190
train==0: 141	, train=1: 238	, ratio: 0.59
test==0:  71	, test=1:  119	, ratio: 0.60
######################(2)######################
train.length: 380	, 	test.length: 189
train==0: 142	, train=1: 238	, ratio: 0.60
test==0:  70	, test=1:  119	, ratio: 0.59
loop: 2


##### 分组交叉验证 GroupKFold
KFold的变体，确保有一个 group 在测试和训练集中都不被表示。通过留出一组特定的不属于测试集和训练集的数据，来测试训练的模型在未知 group 上的性能。需要设定组。

**sklearn参数**：  
- `n_splits`：折数
- `shuffle`：拆分前是否对数据进行混洗

**注**：参数未全部列出，详细可见参考文档

In [190]:
GKF = GroupKFold(n_splits=3)
groups = np.array([random.randint(0,2) for i in range(Xbc.shape[0])])
for loop, (train_idx, test_idx) in enumerate(GKF.split(Xbc, ybc, groups)):
    if loop>10000:
        break
    X_train, X_test = Xbc[train_idx], Xbc[test_idx]
    y_train, y_test = ybc[train_idx], ybc[test_idx]
    a,b = len(y_train[y_train==0]),len(y_train[y_train==1])
    c,d = len(y_test[y_test==0]),len(y_test[y_test==1])
    print('######################(%d)######################' % loop)
    print('train.length: %d\t, \ttest.length: %d' % (train_idx.shape[0], test_idx.shape[0]))
    print('train==0: %d\t, train=1: %d\t, ratio: %.2f' % (a,b,a/b))
    print('test==0:  %d\t, test=1:  %d\t, ratio: %.2f' % (c,d,c/d))
else:
    print('loop:',loop+1)

######################(0)######################
train.length: 367	, 	test.length: 202
train==0: 137	, train=1: 230	, ratio: 0.60
test==0:  75	, test=1:  127	, ratio: 0.59
######################(1)######################
train.length: 384	, 	test.length: 185
train==0: 143	, train=1: 241	, ratio: 0.59
test==0:  69	, test=1:  116	, ratio: 0.59
######################(2)######################
train.length: 387	, 	test.length: 182
train==0: 144	, train=1: 243	, ratio: 0.59
test==0:  68	, test=1:  114	, ratio: 0.60
loop: 3


#### 特征选择：Feature Selection
**参考资料**：
- [机器学习：特征选择（feature selection）](https://blog.csdn.net/qq_33876194/article/details/88403394?utm_medium=distribute.pc_relevant_t0.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-1.control&dist_request_id=&depth_1-utm_source=distribute.pc_relevant_t0.none-task-blog-2%7Edefault%7EBlogCommendFromMachineLearnPai2%7Edefault-1.control)

##### 最优子集选择
子集的数量为$2^p$，计算效率低下且需要很高的计算内存
- **步骤**：
    1. 记不含任何特征的模型为$M_0$，计算这个$M_0$的`测试误差`。                              
    2. 在$M_0$基础上增加一个变量，计算p个模型的RSS，选择RSS最小的模型记作$M_1$，并计算该模型$M_1$的`测试误差`。
    3. 再增加变量，计算p-1个模型的RSS，并选择RSS最小的模型记作$M_2$，并计算该模型$M_2$的`测试误差`。       
    4. 重复以上过程知道拟合的模型有p个特征为止，并选择p+1个模型$\{M_0,M_1,...,M_p \}$中`测试误差`最小的模型作为最优模型。
    
  **注**：`测试误差`值测试集上的误差，指标自选，如：MSE等。

感觉这个方法写起来复杂，运算起来资源也消耗的大，不是很好。不准备写了

In [139]:
def sample_select_best_subset(X:np.array, y:np.array, error:'func'=MSE, random_stat=None):
    pass

##### 向前逐步选择
最优子集选择的优化
- **步骤**：
    1. 记不含任何特征的模型为$M_0$，计算这个$M_0$的测试误差。                    
    2. 在$M_0$基础上增加一个变量，计算p个模型的RSS，选择RSS最小的模型记作$M_1$，并计算该模型$M_1$的测试误差。   
    3. 在最小的RSS模型下继续增加一个变量，选择RSS最小的模型记作$M_2$，并计算该模型$M_2$的测试误差。             
    4. 以此类推，重复以上过程知道拟合的模型有p个特征为止，并选择p+1个模型$\{M_0,M_1,...,M_p \}$中测试误差最小的模型作为最优模型。   
        
  **注**：`测试误差`指测试集上的误差，指标自选，如：MSE等。

In [16]:
def sample_forward_select(X, y, error_func:'func'=MSE, model:'func'=Lasso, random_state=None):
    variate_lst = list(map(str,range(X.shape[1]))) # 将位置作为特征名
    best_idx_lst = [] # 
    best_error_lst = []
    best_model_lst = []
    model = model()
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,
                                                        random_state=random_state)
    M0 = y_test.copy()
    M0[:] = y_test.mean() # 第一个模型使用0个特征，即使用均值模型进行拟合得到
    best_error_lst.append(error_func(M0, y_test)) # M0对应的误差
    while variate_lst: # 当还有特征时继续进行
        best_error = float('inf')
        for variate in variate_lst: # 从剩下的特征里按顺序选特征
            select_idx = best_idx_lst + [int(variate)] # 当前的特征索引序列
            select_data, select_target = X_train[:, select_idx], X_test[:, select_idx] 
            y_pred = model.fit(select_data, y_train).predict(select_target)
            error = error_func(y_test, y_pred) # 得到测试误差
            if error < best_error:
                best_error = error
                best_idx = int(variate)
        else:
            best_idx_lst.append(best_idx)
            best_error_lst.append(best_error)
            best_model_lst.append(model)
            variate_lst.remove(str(best_idx))
    else:
        # 这里写的比较复杂。实际上就是得到最小error，同时得到最小error对应的model
        best_error,best_model = sorted(list(zip(best_error_lst,best_model_lst)),key=lambda x:x[0])[0]
        topk = best_error_lst.index(best_error) # 找到选取几个特征合适
        best_select_variate = best_idx_lst[:topk-1] # 选取topk-1个最合适的特征(因为M0在最前面，需要减去)
        return best_select_variate,best_error,best_model

In [17]:
# model:Lasso, error:MSE
best_select_variate, best_error, best_model = sample_forward_select(Xb,yb,random_state=320)
print('best features:\t',' | '.join(boston_data.columns[best_select_variate]))
print('best error:\t %.5f' % best_error)

best features:	 LSTAT | PTRATIO | RM | CRIM | RAD | INDUS | DIS | ZN
best error:	 17.22996


In [18]:
# model:Ridge, error:MSE
best_select_variate, best_error, best_model = sample_forward_select(Xb,yb,random_state=320,
                                                                   model=Ridge, error_func=MSE)
print('best features:\t',' | '.join(boston_data.columns[best_select_variate]))
print('best error:\t %.5f' % best_error)

best features:	 LSTAT | PTRATIO | RM | DIS | NOX | ZN | RAD | TAX
best error:	 17.91369


In [19]:
# model:Lasso, error:MAE
best_select_variate, best_error, best_model = sample_forward_select(Xb,yb,random_state=320,
                                                                   model=Lasso, error_func=MAE)
print('best features:\t',' | '.join(boston_data.columns[best_select_variate]))
print('best error:\t %.5f' % best_error)

best features:	 LSTAT | PTRATIO | RM | B | RAD | CHAS | NOX | INDUS | ZN | DIS
best error:	 3.07599


In [20]:
# model:Ridge, error:MAE
best_select_variate, best_error, best_model = sample_forward_select(Xb,yb,random_state=320,
                                                                   model=Ridge, error_func=MAE)
print('best features:\t',' | '.join(boston_data.columns[best_select_variate]))
print('best error:\t %.5f' % best_error)

best features:	 LSTAT | PTRATIO | RM | B | CRIM | TAX | NOX | DIS | INDUS
best error:	 3.05965


## Day3

### 压缩估计

#### 岭回归(L2)
![./Image/Ridge.svg](./Image/Ridge.svg)
**参考资料**：
- [sklearn 中的线性回归、岭回归、Lasso回归参数配置及示例](https://blog.csdn.net/VariableX/article/details/107166602)
- [用scikit-learn和pandas学习Ridge回归](https://www.cnblogs.com/pinard/p/6023000.html)
- [从Lasso开始说起](https://zhuanlan.zhihu.com/p/46999826)
- [手写算法-python代码实现Ridge(L2正则项)回归](https://blog.csdn.net/weixin_44700798/article/details/110738525)

##### sklearn调用
**sklearn参数**：
- `alpha`: 正则项系数。数值越大，则对复杂模型的惩罚力度越大。
        调参方法：
        1. 给定alpha较小的值，例如0.1。
        2. 根据验证集准确率以10倍为单位增大或者减小参数值。
        3. 在找到合适的数量级后，在此数量级上微调。
        合适的候选值：[0.001, 0.01, 0.1, 1, 10, 100]
- `normalize`: 是否对各个特征进行标准化。（默认方式：减去均值并除以l2范数）
        标准化的好处:
        1. 加速收敛
        2. 提升精度
- `fit_intercept`: 是否计算截距。
- `solver`: 解决优化问题的算法
    - `svd`: 采⽤用奇异值分解的方法来计算
    - `cholesky`: 采⽤用scipy.linalg.solve函数求得闭式解。
    - `sparse_cg`: 采⽤用scipy.sparse.linalg.cg函数来求取最优解。
    - `lsqr`: 使用scipy.sparse.linalg.lsqr 求解，它是最快的。
    - `sag`: 使用随机平均梯度下降，当n_samples和n_features都较大时，通常比其他求解器更快。
- `max_iter`: 最大迭代次数。

In [5]:
model_L2 = Ridge()
model_L2.fit(X_train, y_train)
y_pred = model_L2.predict(X_test)
print('MSE:\t%.5f' % MSE(y_test, y_pred))

MSE:	30.31898


##### 简单实现
涉及梯度下降法，暂时还不在学习计划内，或许休息期会补上。现在不搞但以后总会搞的	&#x1F609;

    求解方法：
    1. 梯度下降法
    2. 标准方程法
    
**注**:这里准备把最优化算法的理论学习和代码实现放在仓库的同一文件夹下，可前往查看文件：[【扩展1 最优化算法理论及实现】](https://github.com/chenjiyan2001/Datawhale-teamlearning-cHEn/blob/main/2021/03/EnsembleLearning/%E6%89%A9%E5%B1%951%20%E6%9C%80%E4%BC%98%E5%8C%96%E7%AE%97%E6%B3%95%E7%90%86%E8%AE%BA%E5%8F%8A%E5%AE%9E%E7%8E%B0.ipynb)。计划在本次学习结束后集中时间学习。

In [ ]:
class sample_ridge():
    def __init__(self, alpha=0.1, max_iter=10000):
        self.alpha = alpha
        self.max_iter = max_iter
my_lasso = sample_ridge(alpha=0.5)       

#### Lasso回归(L1)
![./Image/Lasso.svg](./Image/Lasso.svg)
**参考资料**：
- [sklearn 中的线性回归、岭回归、Lasso回归参数配置及示例](https://blog.csdn.net/VariableX/article/details/107166602)
- [Lasso回归算法： 坐标轴下降法与最小角回归法小结](https://www.cnblogs.com/pinard/p/6018889.html)
- [手写算法-python代码实现Lasso回归](https://blog.csdn.net/weixin_44700798/article/details/110690015)
- [从Lasso开始说起](https://zhuanlan.zhihu.com/p/46999826)

##### sklearn调用
**sklearn参数**：
- `alpha`: 正则项系数。数值越大，则对复杂模型的惩罚力度越大。
        调参方法：
        1. 给定alpha较小的值，例如0.1。
        2. 根据验证集准确率以10倍为单位增大或者减小参数值。
        3. 在找到合适的数量级后，在此数量级上微调。
        合适的候选值：[0.001, 0.01, 0.1, 1, 10, 100]
- `normalize`: 是否对各个特征进行标准化。（默认方式：减去均值并除以l2范数）
        标准化的好处:
        1. 加速收敛
        2. 提升精度
- `fit_intercept`: 是否计算截距。
- `max_iter`: 最大迭代次数。
- `selection`: 指定了每轮迭代时，选择权重向量的哪个分量来更新
    - `random`: 更新的时候，随机选择权重向量的⼀个分量来更更新。
    - `cyclic`: 更新的时候，从前向后依次选择权重向量的⼀个分量来更新。

**注**：参数未全部列出，详细可见参考文档

In [53]:
model_L1 = Lasso()
model_L1.fit(X_train, y_train)
y_pred = model_L1.predict(X_test)
print('MSE:\t%.5f' % MSE(y_test, y_pred))

MSE:	34.17221


##### 简单实现
暂时还不会。。慢慢来吧

    求L1范数的损失函数极小值的解法：
    1. 坐标轴下降法(coordinate descent)
    2. 最小角回归法(Least Angle Regression,LARS)

In [9]:
class sample_lasso():
    def __init__(self, alpha=0.1, max_iter=10000):
        self.alpha = alpha
        self.max_iter = max_iter
my_lasso = sample_lasso(alpha=0.5)        

## Day4

### 降维
**参考资料**：
- [各类降维方法总结](https://www.jianshu.com/p/75e805ff247c?utm_campaign=maleskine&utm_content=note&utm_medium=seo_notes&utm_source=recommendation)

#### 主成分分析 PCA

##### sklearn调用

In [69]:
# 保留5维
pca = PCA(n_components=5)
X = pca.fit_transform(Xb)
print('特征维数：\t\t',pca.n_components_)
print('保留成分方差之和：\t\t',sum(pca.explained_variance_))
print('保留成分方差百分比之和：\t',sum(pca.explained_variance_ratio_))
print('噪声方差：\t\t', pca.noise_variance_)

特征维数：		 5
保留成分方差之和：		 38275.12164583213
保留成分方差百分比之和：	 0.9984806947092307
噪声方差：		 7.28000991024328


In [68]:
# 保留95%
pca = PCA(n_components=0.95)
X = pca.fit_transform(Xb)
print('特征维数：\t\t',pca.n_components_)
print('保留成分方差之和：\t\t',sum(pca.explained_variance_))
print('保留成分方差百分比之和：\t',sum(pca.explained_variance_ratio_))
print('噪声方差：\t\t', pca.noise_variance_)

特征维数：		 2
保留成分方差之和：		 37140.24132221935
保留成分方差百分比之和：	 0.9688751429772733
噪声方差：		 108.46549117223913


In [67]:
# 通过MLE最大似然算法自动选择
pca = PCA(n_components='mle')
X = pca.fit_transform(Xb)
print('特征维数：\t\t',pca.n_components_)
print('保留成分方差之和：\t\t',sum(pca.explained_variance_))
print('保留成分方差百分比之和：\t',sum(pca.explained_variance_ratio_))
print('噪声方差：\t\t', pca.noise_variance_)

特征维数：		 12
保留成分方差之和：		 38333.35867065725
保留成分方差百分比之和：	 0.9999999203185791
噪声方差：		 0.0030544567251183903


##### 简单实现

In [18]:
class sample_pca():
    def __init__(self, n_components=5, threshold=False):
        self.n_components = n_components
        self.threshold = threshold
        
    def fit_transform(self,X):
        X_std = (X-np.mean(X,0))/np.std(X,0,ddof=1) # 标准化
        C = np.cov(X_std,rowvar=0) # 求协方差矩阵
        tzz, tzxl = np.linalg.eig(C) # 求得特征值和特征向量
        ind = np.argsort(-tzz) # 排序
        CPV = np.cumsum(tzz[ind])/np.sum(tzz) # 求得累积方差贡献率CPV
        if self.threshold: # 参数threshold有值时通过threshold选取特征维数
            self.n_components = np.sum(CPV < self.threshold)+1
        loading = tzxl[:,ind][:,0:self.n_components] # 取得特征向量中与较大特征值对应的部分
        new_X = X_std.dot(loading) # 计算新的矩阵
        return new_X

In [16]:
# n_components = 2
my_pca = sample_pca(n_components = 2)
my_pca.fit_transform(Xb)

array([[ 2.09622303, -0.77234843],
       [ 1.45581099, -0.59139995],
       [ 2.07254655, -0.59904658],
       ...,
       [ 0.31205166, -1.15410433],
       [ 0.27025162, -1.04033206],
       [ 0.12567884, -0.76122473]])

In [19]:
# threshold = 0.95
my_pca = sample_pca(threshold = 0.95)
my_pca.fit_transform(Xb)

array([[ 2.09622303, -0.77234843, -0.34260368, ..., -0.31832574,
        -0.29553933,  0.42451671],
       [ 1.45581099, -0.59139995,  0.69451201, ..., -0.55331369,
         0.22344881,  0.16679701],
       [ 2.07254655, -0.59904658, -0.16695638, ..., -0.48408091,
        -0.10506216, -0.06970615],
       ...,
       [ 0.31205166, -1.15410433,  0.40819364, ..., -0.29382858,
         0.63802897, -0.98006238],
       [ 0.27025162, -1.04033206,  0.58487527, ..., -0.27132856,
         0.57877171, -0.9358292 ],
       [ 0.12567884, -0.76122473,  1.29360184, ..., -0.17513633,
         0.13325011, -0.85384425]])

#### 其他降维方法

##### LDA

In [73]:
# 这个比较花时间诶
lda = LDA(n_components=5, max_iter=1000)
X = lda.fit_transform(Xbc, ybc)
# sum(lda.scalings_)

##### 奇异值分解 SVD

In [82]:
svd = TruncatedSVD()
svd.fit_transform(X)
sum(svd.explained_variance_ratio_)

0.9999999999999871